## A feed-forward Neural Network for secondary structure prediction
This notebook uses a Neural Network based on code from Stevens and Boucher (2014, Python for Biology CUP). A Predictive network is trained using data on known secondary structure of k-mers of 5 amino-acids taken from a set of PDB structures. Three secondary structure states are defined: H,C, and S. H and S are helix and strand respectively while C is for coil which is a range of structures not with regular H-bonding pattern. 

Secondary structure is much more complicated than indicated by the simple classification of this data - full details are available from the analysis of the H-bonding arrangements. The program DSSP (https://swift.cmbi.umcn.nl/gv/dssp/DSSP_3.html) is a well-tested approach to this problem. This produces a description of the secondary structure in a known protein structure. For historical reasons DSSP uses E for Strands. A related DSSR program gives RNA secondary structure.

It is useful to be able to predict the secondary structure of a protein for which there is only sequence available. One approach would be to align it with a known homologous structure. The approach here is to use the sequence in the neighbourhood of a residue as a basis for a neural network prediction. 

The network here is a simple three layer feed-forward one. The number of nodes in the hidden layer can be defined by the programmer. But the number of input and output nodes is defined by the sizes of the input and output data vectors.


In [1]:
##Run this cell to import numpy 
from numpy import tanh, ones, append, array, zeros, random, sum

The neural network function takes input data for the first layer of Network nodes, applies the the first weighted connections to pass the signal to the hidden layer of nodes, then applies the second weights to produce output. 

The output may not be optimized as the function also operates on the weighting during training. However after training the function gives predictions so takes its name from that. 

The weightsIn values define the strength of connection between the input nodes and the hidden nodes. Similarly weightsOut define the strengths of connection between the hidden and the output nodes. 

The wieghts are given as matrices with the rows indexing the nodes in a layer and the columns indexing the nodes in the other layer. 

The signalIn vector is the input features and an extra value of 1.0. This additional value is called the bias node which is used to tune the baseline response of the network. The baseline is the level without a meaningful signal. 

Setting the bias node value happens during training to adapt to the values in the input data. This means the input data don't need to pre-prepared with a mean of zero.

In [2]:
##Run this cell to define the function
def neuralNetPredict(inputVec, weightsIn, weightsOut):
    """ uses the current weights in a neural network
    to make a prediction from an input vector
    all input and output are numpy data structures""" 
    signalIn = append(inputVec, 1.0) # input layer

    prod = signalIn * weightsIn.T
    sums = sum(prod, axis=1)
    signalHid = tanh(sums)    # hidden    layer

    prod = signalHid * weightsOut.T
    sums = sum(prod, axis=1)
    signalOut = tanh(sums)    # output    layer

    return signalIn, signalHid, signalOut
 

The .T function gives the transpose of a matrix - that is the matrix with the columns turned into rows and the rows turned into columns. 

This is used so that the input signal gets applied to all the hidden nodes.

The network applies the hyperbolic tangent function (tanh) to get the signal output from all the nodes in layer. Hyperbolic tan is a sigmoidal function that varies from -1 to 1, so it is much better than ordinary tan that runs off to infinity. The tanh function defines the output of that node given an input or set of inputs. As a nod to the output of neurones, which depends on an activation level across their cell membrane, the output function is called the *Activation* function.

In operation only the signalOut from the output layer is of interest. But during training the response signals from the other layers also need to adjust the weighting scheme.

### Training 
The weighting scheme (and gain) will be optimized by using a training dataset. 

The training data will be an input feature vector and a known output vector. The order of the data will be randomly shuffled to avoid bias. The number of hidden nodes needs to be specified and the number of optimization cycles. 

After each cycle the 'error' between the output signal of the network and the known training set output is used to adjust the network weights. The difference is combined with the *gradient* in the signal values - calculated from the tanh activation function (conveniently the gradient of tanh(sig) is 1-sig^2 or 1 - (sig x sig)). 

Early in training large difference can make the network go haywire so the speed of weight changing is damped down by a 'rate' and 'momentum' multipliers (usually the default values of 0.5 and 0.2 are good enough. 

More damping would mean that many more cycles would be needed for the weights to converge. 

The training will work back from the value of the error to adjust the weighting scheme of the network. This is called *back propagation*. 

The use of the gradient is crucial as it means initial adjustments will be large but then finer adjustments will be made as the optimum is approached.

In [3]:
##Run this cell to define the function
def neuralNetTrain(trainData, numHid, steps=100, rate=0.5, momentum=0.2):
    """ uses training data to set the weights in a simple
    neural network, number of hidden nodes is specified"""
    numInp = len(trainData[0][0])
    numOut = len(trainData[0][1])
    numInp += 1
    minError = None

    sigInp = ones(numInp)
    sigHid = ones(numHid)
    sigOut = ones(numOut)

    wInp = random.random((numInp, numHid))-0.5
    wOut = random.random((numHid, numOut))-0.5
    bestWeightMatrices = (wInp, wOut)

    cInp = zeros((numInp, numHid))
    cOut = zeros((numHid, numOut))

    for x, (inputs, knownOut) in enumerate(trainData):
        trainData[x] = (array(inputs), array(knownOut))
 
    for step in range(steps):  
        random.shuffle(trainData) # Important to avoid bias
        error = 0.0
 
        for inputs, knownOut in trainData:
            sigIn, sigHid, sigOut = neuralNetPredict(inputs, wInp, wOut)

            diff = knownOut - sigOut
            error += sum(diff * diff)

            gradient = ones(numOut) - (sigOut*sigOut)
            outAdjust = gradient * diff 

            diff = sum(outAdjust * wOut, axis=1)
            gradient = ones(numHid) - (sigHid*sigHid)
            hidAdjust = gradient * diff 

            # update output 
            change = outAdjust * sigHid.reshape(numHid, 1)
            wOut += (rate * change) + (momentum * cOut)
            cOut = change
 
            # update input 
            change = hidAdjust * sigIn.reshape(numInp, 1)
            wInp += (rate * change) + (momentum * cInp)
            cInp = change
 
        if (minError is None) or (error < minError):
            minError = error
            bestWeightMatrices = (wInp.copy(), wOut.copy())
            print("Step: %d Error: %f" % (step, error))
    
    return bestWeightMatrices

### Testing the functions
Simple data to test a network can be binary input vectors with the desired output being an 'exclusive OR' (EOR) response. This responds True if any input is true but False is both are together. 

In [4]:
##Run this cell to define the test data
testEORdata = [[[0,0], [0]],[[0,1], [1]], [[1,0], [1]], [[1,1], [0]]]

The network test uses two hidden nodes - in real use several values would be tried to find the best performance.
Run the cell below to see if the training converges.

In [5]:
##Run this cell to train the network
wMatrixIn, wMatrixOut = neuralNetTrain(testEORdata, 2, 1000)

Step: 0 Error: 1.662945
Step: 1 Error: 1.216083
Step: 3 Error: 1.159888
Step: 10 Error: 1.136167
Step: 20 Error: 1.102602
Step: 21 Error: 1.025619
Step: 22 Error: 0.999384
Step: 25 Error: 0.934159
Step: 28 Error: 0.933513
Step: 30 Error: 0.910591
Step: 31 Error: 0.892805
Step: 39 Error: 0.887306
Step: 41 Error: 0.869018
Step: 45 Error: 0.849426
Step: 52 Error: 0.839845
Step: 54 Error: 0.833483
Step: 59 Error: 0.832764
Step: 60 Error: 0.818643
Step: 61 Error: 0.805045
Step: 64 Error: 0.799074
Step: 66 Error: 0.784809
Step: 67 Error: 0.773271
Step: 70 Error: 0.754645
Step: 71 Error: 0.705676
Step: 76 Error: 0.672309
Step: 77 Error: 0.649118
Step: 81 Error: 0.606411
Step: 82 Error: 0.593129
Step: 86 Error: 0.536180
Step: 88 Error: 0.419359
Step: 90 Error: 0.378517
Step: 96 Error: 0.348409
Step: 99 Error: 0.337254
Step: 101 Error: 0.331303
Step: 102 Error: 0.263926
Step: 103 Error: 0.215745
Step: 105 Error: 0.200334
Step: 107 Error: 0.184302
Step: 108 Error: 0.153578
Step: 110 Error: 0.134

Here quite good convergence has occurred and there is no oscillation. Perhaps you can see that the initial steps are giving large changes in the error while later on there are smaller and smaller changes. This is owing to the effect of the gradient calculation. The changes in the actual weights are not printed, but will follow the same trend.

The output weight matrices can then be run on test data for evaluation. Test data should be inputs with known output but which were not included in the training set. 

Obviously it is not possible to give any new data for the EOR function as the training set covered all possible responses!

But the trained network should be able to do a reasonable job on the training set.
Run the following cell to compare the output of the network with the actual values of the training set outputs.

In [7]:
##Run this cell to test the network
for inputs, knownOut in testEORdata:
    sIn, sHid, sOut =    neuralNetPredict(array(inputs), wMatrixIn, wMatrixOut)
    print(knownOut, sOut[0])

[1] 0.9834937071829389
[1] 0.9833967025093591
[0] 0.011231149515516878
[0] -0.00048119258000472944


### Simple feature vectors for sequence data
A simple numbering scheme is used to convert to the sequence alphabet to a numeric form as an input vector. For proteins that is number from 1 to 20 from the list of one-letter codes.

k-mers with k=5 are used. Only the output for the middle residue is required but the network will use the neighbours to predict the secondary structure of the middle one.  

Although static k-mer are used for training in practice a prediction in a moving 5-mer window could be implemented. 

The possible outputs are also coded as integers for the more restricted alphabet of H, C, and S

In [10]:
##Run this cell to define the dictionaries for the vectors
aminoAcids = 'ACDEFGHIKLMNPQRSTVWY'
aaIndexDict = {}
for i, aa in enumerate(aminoAcids):
        aaIndexDict[aa] = i

ssIndexDict = {}
ssCodes = 'HCS'
for i, code in enumerate(ssCodes):
        ssIndexDict[code] = i

Here is a very limited set of training data. It shows the raw format which is a 5-mer string and the secondary structure that was observed for the central residue of this in at least one PDB structure. 

The actual structure is a simplified output from the DSSP program mentioned in the introduction. DSSP acutally distinguishes more structures that the three here - for example there are other kinds of helix. But these complications are not dealt with.

In [11]:
##Run this cell to define the training set
seqSecStrucData = [('ADTLL','S'),
                         ('DTLLI','S'),
                         ('TLLIL','S'),
                         ('LLILG','S'),
                         ('LILGD','S'),
                         ('ILGDS','S'),
                         ('LGDSL','C'),
                         ('GDSLS','H'),
                         ('DSLSA','H'),
                         ('SLSAG','H'),
                         ('LSAGY','H'),
                         ('SAGYR','C'),
                         ('AGYRM','C'),
                         ('GYRMS','C'),
                         ('YRMSA','C'),
                         ('RMSAS','C')]

The training data has to be converted to the numerical code. Here is a function to to that.

In [12]:
##Run this cell to define the function
def convertSeqToVector(seq, indexDict):
    """converts a one-letter sequence to numerical
    coding for neural network calculations"""   
    numLetters = len(indexDict)
    vector = [0.0] * len(seq) * numLetters

    for pos, letter in enumerate(seq):
        index = pos * numLetters + indexDict[letter]    
        vector[index] = 1.0

    return vector


The training data is prepared with this.

In [13]:
##Run this cell to create the training data
trainingData = []
for seq, ss in seqSecStrucData:
 
        inputVec = convertSeqToVector(seq, aaIndexDict)
        outputVec = convertSeqToVector(ss, ssIndexDict)
 
        trainingData.append( (inputVec, outputVec) )


And then the network is trained. Here there are 3 hidden nodes specified.

In [14]:
   wMatrixIn, wMatrixOut = neuralNetTrain(trainingData, 3, 1000)

Step: 0 Error: 18.544195
Step: 1 Error: 13.273331
Step: 2 Error: 10.049923
Step: 3 Error: 8.081151
Step: 4 Error: 6.679041
Step: 5 Error: 4.146071
Step: 6 Error: 3.434692
Step: 7 Error: 1.841507
Step: 9 Error: 0.771946
Step: 10 Error: 0.557903
Step: 11 Error: 0.451036
Step: 12 Error: 0.264221
Step: 13 Error: 0.189702
Step: 15 Error: 0.142910
Step: 18 Error: 0.138171
Step: 20 Error: 0.112035
Step: 22 Error: 0.083567
Step: 24 Error: 0.058639
Step: 26 Error: 0.053782
Step: 31 Error: 0.047820
Step: 33 Error: 0.041850
Step: 37 Error: 0.041696
Step: 41 Error: 0.034369
Step: 42 Error: 0.024654
Step: 62 Error: 0.016462
Step: 80 Error: 0.011997
Step: 110 Error: 0.008466
Step: 111 Error: 0.007629
Step: 116 Error: 0.006890
Step: 146 Error: 0.006702
Step: 148 Error: 0.005243
Step: 164 Error: 0.004507
Step: 198 Error: 0.004341
Step: 199 Error: 0.004154
Step: 200 Error: 0.003424
Step: 250 Error: 0.003408
Step: 281 Error: 0.003361
Step: 287 Error: 0.003207
Step: 299 Error: 0.002994
Step: 350 Error: 0

You will see that this training has converged nicely. The only problem is that it was for a very restricted set of sequence data. 

There are 3 x 20 = 60 theoretical combinations of amino acid residues with secondary structure states. But for particular residues some of these are favoured and others disfavoured. 

For each residue there will be 20^4 = 160 000 different contexts that then could possibly occur in. Although some of the resulting 5-mers are actually quite rare in structured proteins. 

All the same, it would be good to have a larger training set. It is better to have some rare examples of residue state combinations although, of course, the network will not have to predict them frequently.

One thing to remember is to retain some test examples - where the answer is known but which are not in the training set. 

In its current, poorly-trained state, the network is still able to make a reasonable predictions. But only if the test is clearly related to examples that it has seen. testSecStrucSeq here is very similar to examples in the seqSecStrucData training set.

In [15]:
##Run this cell to test for a new 5-mer
testSecStrucSeq = 'DLLSA'
testSecStrucVec = convertSeqToVector(testSecStrucSeq, aaIndexDict)
testSecStrucArray = array( [testSecStrucVec,] )

sIn, sHid, sOut =    neuralNetPredict(testSecStrucArray, wMatrixIn, wMatrixOut)
index = sOut.argmax()
print("Test prediction: %s" % ssCodes[index])

Test prediction: S


### Inporting a larger training set
A much larger training set is provided as a comma separated file called:
"PDB_protein_secondary_5mers.csv". 

    from csv import reader #may help here
Can you create a training set vector data structure from this and use it to train the network?
If you would like some examples of test data, here are some examples from the recently-determined PDB 6aam.pdb

    S: EMVAV, KVSKY, YKGCC
    H: LAQLL, ICEGM, ASVDW
    C: ERLPR, GDFGL, YKFYY
